In [4]:
# 데이터 다운로드
# !kaggle competitions download -c word2vec-nlp-tutorial


  0%|          | 0.00/51.7M [00:00<?, ?B/s]
  2%|1         | 1.00M/51.7M [00:00<00:14, 3.65MB/s]
  4%|3         | 2.00M/51.7M [00:00<00:13, 3.84MB/s]
  6%|5         | 3.00M/51.7M [00:00<00:11, 4.31MB/s]
  8%|7         | 4.00M/51.7M [00:00<00:11, 4.37MB/s]
 10%|9         | 5.00M/51.7M [00:01<00:10, 4.55MB/s]
 12%|#1        | 6.00M/51.7M [00:01<00:11, 4.26MB/s]
 14%|#3        | 7.00M/51.7M [00:01<00:10, 4.35MB/s]
 15%|#5        | 8.00M/51.7M [00:01<00:09, 4.60MB/s]
 17%|#7        | 9.00M/51.7M [00:02<00:09, 4.76MB/s]
 19%|#9        | 10.0M/51.7M [00:02<00:09, 4.52MB/s]
 21%|##1       | 11.0M/51.7M [00:02<00:08, 5.14MB/s]
 23%|##3       | 12.0M/51.7M [00:02<00:08, 5.05MB/s]
 25%|##5       | 13.0M/51.7M [00:02<00:07, 5.39MB/s]
 27%|##7       | 14.0M/51.7M [00:02<00:06, 5.82MB/s]
 29%|##9       | 15.0M/51.7M [00:03<00:06, 6.10MB/s]
 31%|###       | 16.0M/51.7M [00:03<00:06, 5.86MB/s]
 33%|###2      | 17.0M/51.7M [00:03<00:06, 5.78MB/s]
 35%|###4      | 18.0M/51.7M [00:03<00:05, 6.01MB/s]
 

### 필요한 모듈 import

In [7]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

warnings.filterwarnings('ignore')

### 데이터 불러오기

- id - Unique ID of each review
- sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
- review - Text of the review

In [9]:
data = "./word2vec-nlp-tutorial/"

In [15]:
train = pd.read_csv(os.path.join(data, 'labeledTrainData.tsv'), delimiter='\t') # tsv 파일
test = pd.read_csv(os.path.join(data, 'testData.tsv'), delimiter='\t')
unlabeled_train = pd.read_csv(os.path.join(data, 'unlabeledTrainData.tsv'), delimiter='\t', error_bad_lines = False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [16]:
print(train.shape)
train.head()

(25000, 3)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [17]:
print(test.shape)
test.head()

(25000, 2)


,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [18]:
print(unlabeled_train.shape)
unlabeled_train.head()

(49998, 2)


,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


### 전처리
- html태그, 불용어, 특수기호 등 제거

In [36]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

sample 데이터로 프로세스 확인 후 함수로 묶어주기

In [49]:
sample = train['review'][0]

In [24]:
# 특수기호 제거
import re

In [41]:
# 불용어 사전
eng_stopwords = stopwords.words('english')

In [50]:
def preprocessing(sentence):
    soup = BeautifulSoup(sentence, 'html.parser')
    cleaned = re.sub('[^a-zA-Z]',' ', soup.text)
    cleaned =  cleaned.lower()
    cleaned = [word for word in cleaned.split() if word not in eng_stopwords]
    return ' '.join(cleaned) # 단어를 문장으로 결합하여 리턴

In [54]:
all_review = pd.concat([train['review'], test['review'], unlabeled_train['review']])
all_review_clean = all_review.apply(preprocessing)

In [55]:
all_review_clean.head()

0    stuff going moment mj started listening music ...
1    classic war worlds timothy hines entertaining ...
2    film starts manager nicholas bell giving welco...
3    must assumed praised film greatest filmed oper...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

#### CountVectorizer
`sklearn.feature_extraction.text.CountVectorizer`
- 단어 사전을 만든 뒤 해당 단어에 대한 빈도 수 count

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

In [58]:
cv = CountVectorizer(analyzer='word', max_features=5000)
all_review_cv = cv.fit_transform(all_review_clean)
all_review_cv.shape

(99998, 5000)

In [60]:
all_review_cv.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### train test split

In [61]:
train_sentences = all_review_cv[:len(train)]
test_sentences = all_review_cv[-len(test):]
train_sentences.shape, test_sentences.shape

((25000, 5000), (25000, 5000))

In [63]:
train_labels = train['sentiment']
train_labels.shape

(25000,)

### Model

In [64]:
from sklearn.ensemble import RandomForestClassifier

In [65]:
rfc = RandomForestClassifier(n_estimators=1000, max_depth=8, n_jobs=-1) # max_depth : 과대적합 방지

In [66]:
rfc.fit(train_sentences,train_labels)
prediction = rfc.predict(test_sentences)
prediction.shape

(25000,)

### Submission

In [68]:
submission = pd.read_csv(os.path.join(data,'sampleSubmission.csv'))
submission.head()

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0


In [69]:
submission['sentiment'] = prediction
submission['sentiment'].value_counts()
# 긍정 13792개, 부정 11208개로 예측함

1    13792
0    11208
Name: sentiment, dtype: int64

In [72]:
import datetime

In [77]:
timestring = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [79]:
filename = f'submission/submission-{timestring}.csv'

In [81]:
submission.to_csv(filename, index=False)